In [ ]:
!pip install pykrx
!pip install openai

In [ ]:
from pykrx import stock
import pandas as pd
import matplotlib.pyplot as plt

# 삼성전자의 주식 코드
stock_code = "005930"

# 특정 기간 동안의 주가 데이터를 가져오기
start_date = "20230301"  # 시작 날짜 예시
end_date = "20230331"    # 종료 날짜 예시
df = stock.get_market_ohlcv_by_date(start_date, end_date, stock_code)

# 데이터 확인
print(df.head())

In [ ]:
# system prompt에 출력 형식을 지정

import os
from openai import OpenAI
from pykrx import stock
import json
import pandas as pd

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

# 함수 정의
def get_stock_info(stock_code, start_date, end_date):
    stock_info = {
        "stock_code": stock_code,
        "start_date": start_date,
        "end_date": end_date,
    }

    df = stock.get_market_ohlcv_by_date(start_date, end_date, stock_code)

    return df.to_json()


messages = [{"role": "user", "content": "삼성전자의 2020년도의 주가 정보를 분석해줘."}]
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_stock_info",
            "description": "특정 주식의 일정 기간동안의 정보를 분석해줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "stock_code": {
                        "type": "string",
                        "description": "주식 코드",
                    },
                    "start_date": {
                        "type": "string",
                        "description": "시작일, 날짜 포맷은 yyyyMMdd"
                    },
                    "end_date": {
                        "type": "string",
                        "description": "종료일, 날짜 포맷은 yyyyMMdd"
                    }
                },
                "required": ["stock_code", "start_date", "end_date"],
            },
        }
    }
]

# 첫번째 메시지
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=tools,
    tool_choice="auto" if tools is not None else None
    )
response_message = response.choices[0].message
print(response_message)

# 함수 호출
available_functions = {
    "get_stock_info": get_stock_info,
}

for tool_call in response_message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        stock_code=function_args['stock_code'],
        start_date=function_args['start_date'],
        end_date=function_args['end_date'],
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "function",
            "name": function_name,
            "content": function_response
        }
    )
    messages.append(
        {
          "role": "system",
          "content": "당신은 주식 종목 분석가입니다. 주어진 종목 정보 데이터를 이용해 사용자에게 주가의 분석해서 분석한 정보를 사용자에게 제공해야 합니다."
        },
    )

# 두번째 메시지    
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)
print(second_response.choices[0].message.content)
